In [1]:
# Import data from Excel sheet
import pandas as pd
df = pd.read_excel('mwp1_CAT12_DELCODE/Antrag 137_Teipel_Regional Connectivity_20190429_Amendment_MDedit.xlsx', sheet_name='Baseline', header=1)
#print(df)
sid = df['Respeudonym']
grp = df['prmdiag']
age = df['age']
sex = df['sex_bin_1female']
tiv = df['TIV_CAT12']
field = df['FieldStrength']
grpbin = (grp > 0) # 0=CN, ...

In [2]:
# Scan for nifti file names
import glob
dataDELCODE_CN = sorted(glob.glob('mwp1_CAT12_DELCODE/0_CN/*.nii.gz'))
dataDELCODE_MCI = sorted(glob.glob('mwp1_CAT12_DELCODE/2_MCI/*.nii.gz'))
dataDELCODE_AD = sorted(glob.glob('mwp1_CAT12_DELCODE/5_AD/*.nii.gz'))
dataFiles = dataDELCODE_CN + dataDELCODE_MCI + dataDELCODE_AD
numfiles = len(dataFiles)
print('Found ', str(numfiles), ' nifti files')

Found  474  nifti files


In [3]:
# Match covariate information
import re
import numpy as np
from pandas import DataFrame
from keras.utils import to_categorical
debug = False
cov_idx = [-1] * numfiles # list; array: np.full((numfiles, 1), -1, dtype=int)
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
  p = [j for j,x in enumerate(dataFiles) if re.search('_%s' % id, x)] # extract ID numbers from filename, translate to Excel row index
  if len(p)==0:
    if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
  else:
    if debug: print('Found %04d in %s: %s' % (id, p[0], dataFiles[p[0]]))
    cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
labels = to_categorical(np.asarray(labels)) # use grps to access original labels
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]

Using TensorFlow backend.


Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  0


In [4]:
# Load residualized data from disk
import h5py
hf = h5py.File('residuals_DELCODE_wb_mwp1_CAT12_MNI.hdf5', 'r')
hf.keys # read keys
images = np.array(hf.get('images'))
hf.close()
print(images.shape)

(474, 100, 100, 120, 1)


In [5]:
import tensorflow as tf
print(tf.__version__)
# disable tensorflow deprecation warnings
import logging
logging.getLogger('tensorflow').disabled=True
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto(
#    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
#    # device_count = {'GPU': 1}
#)
#config.gpu_options.allow_growth = True
#session = tf.Session(config=config)
#set_session(session)

1.15.4


In [6]:
import nibabel as nib

# define FOV to reduce required memory size
x_range_from = 10; x_range_to = 110
y_range_from = 13; y_range_to = 133
z_range_from = 5; z_range_to = 105

fname_overlay = sorted(glob.glob('Hippocampus_masks/aal_*.nii*')) + glob.glob('Hippocampus_masks/brainmask.nii*')
data_overlay = np.zeros((len(fname_overlay), z_range_to-z_range_from, x_range_to-x_range_from, y_range_to-y_range_from, 1), dtype=np.float32) # numfiles× z × x × y ×1; avoid 64bit types

for i in range(len(fname_overlay)):   
    img = nib.load(fname_overlay[i])
    img = img.get_fdata()[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to]
    img = np.transpose(img, (2, 0, 1)) # reorder dimensions to match coronal view z*x*y in MRIcron etc.
    img = np.flip(img) # flip all positions
    data_overlay[i, :,:,:, 0] = np.nan_to_num(img)

In [7]:
###### check which files were loaded #####
print(fname_overlay)
print(data_overlay.shape)

['Hippocampus_masks\\aal_basal_ganglia.nii.gz', 'Hippocampus_masks\\aal_cerebellum.nii.gz', 'Hippocampus_masks\\aal_frontal.nii.gz', 'Hippocampus_masks\\aal_hippocampus.nii.gz', 'Hippocampus_masks\\aal_insula_cingulate.nii.gz', 'Hippocampus_masks\\aal_occipital.nii.gz', 'Hippocampus_masks\\aal_parietal.nii.gz', 'Hippocampus_masks\\aal_temporal.nii.gz', 'Hippocampus_masks\\brainmask.nii.gz']
(9, 100, 100, 120, 1)


In [8]:
# initialize parameters for analyzer
from keras.models import load_model, Model
#!pip install innvestigate
import innvestigate
import innvestigate.utils as iutils
import numpy as np
from matplotlib import pyplot as plt
import scipy
import csv
import os

# see https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb for a list of alternative methods
methods = [ # tuple with method,     params,                  label
#            ("deconvnet",            {},                      "Deconvnet"),
#            ("guided_backprop",      {},                      "Guided Backprop"),
#            ("deep_taylor.bounded",  {"low": -1, "high": 1},  "DeepTaylor"),
#            ("input_t_gradient",     {},                      "Input * Gradient"),
#            ("lrp.z",                {},                      "LRP-Z"),
#            ("lrp.epsilon",          {"epsilon": 1},          "LRP-epsilon"),
#            ("lrp.alpha_1_beta_0",   {"neuron_selection_mode":"index"},     "LRP-alpha1beta0"),
    ("lrp.sequential_preset_a", {"neuron_selection_mode": "index", "epsilon": 1e-10}, "LRP-CMPalpha1beta0"), # LRP CMP rule taken from https://github.com/berleon/when-explanations-lie/blob/master/when_explanations_lie.py
]

# define output csv header manually
header = ['SID', 'sum_relevance_basal_ganglia', 'sum_relevance_cerebellum', 'sum_relevance_frontal', 
          'sum_relevance_hippocampus', 'sum_relevance_insula_cingulate', 'sum_relevance_occipital', 
          'sum_relevance_parietal', 'sum_relevance_temporal', 'sum_relevance_brain']

In [9]:
# Load CNN model from disk and create analyzer

for k in range(10):
    mymodel = load_model('model_checkpoints/resmodel_wb_cv%d.best.hdf5' % (k+1))
    #mymodel.summary()
    #model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
    #model_wo_softmax = Model(inputs=mymodel.inputs,
    #                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
    #                          name=('wo_softmax_cv%d' % (k+1))) 
    #
    mymodel.layers[-1].activation=tf.keras.activations.linear
    mymodel.save('tmp_wo_softmax.hdf5')
    model_wo_softmax = load_model('tmp_wo_softmax.hdf5')
    os.remove('tmp_wo_softmax.hdf5')
    if (k==0):
        model_wo_softmax.summary()

    # create analyzer
    analyzers = []
    for method in methods:
        analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
        # Some analyzers require training.
        #   analyzer.fit(test_img, batch_size=30, verbose=1)
        #  analyzers.append(analyzer)

    with open(('hipp_lrpCMP_resmodel_DELCODE_cv%d.csv' % (k+1)), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(header)
        print('exporting values to hipp_lrpCMP_resmodel_DELCODE_cv%d.csv' % (k+1))
        
        for indx in range(len(grps)): # for all scans
            test_img = images[indx]
            test_img = np.reshape(test_img, (1,)+ test_img.shape) # add first subj index again to mimic original array structure

            #for method,analyzer in zip(methods, analyzers):
            a = analyzer.analyze(test_img, neuron_selection=1)
            np.clip(a, a_min=0, a_max=None, out=a) # set negative values to 0
            #a = np.squeeze(a)
            #a = scipy.ndimage.filters.gaussian_filter(a, sigma=0.8) # smooth activity image
            #scale = np.quantile(a, 0.99) # no need for abs(a)
            #if scale!=0:  # ignore if relevance maps contains only zeros, output will be zero as well
            #    a = (a/scale)

            rel_all = np.multiply(data_overlay, np.repeat(a, len(fname_overlay), axis=0))

            csvwriter.writerow([grps.RID.iloc[indx], ] + np.sum(rel_all, axis=(1,2,3,4)).tolist())
        csvfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 100, 100, 120, 5)  140       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 50, 60, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 60, 5)     20        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 60, 5)     680       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 25, 25, 30, 5)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 30, 5)     20        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 25, 30, 5)     680       
__________

In [10]:
# run again for whole-dataset model
for k in [0]:
    mymodel = load_model('model_checkpoints/resmodel_wb_whole_ds.hdf5')
    #mymodel.summary()
    #model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
    #model_wo_softmax = Model(inputs=mymodel.inputs,
    #                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
    #                          name=('wo_softmax_cv%d' % (k+1))) 
    #
    mymodel.layers[-1].activation=tf.keras.activations.linear
    mymodel.save('tmp_wo_softmax.hdf5')
    model_wo_softmax = load_model('tmp_wo_softmax.hdf5')
    os.remove('tmp_wo_softmax.hdf5')
    if (k==0):
        model_wo_softmax.summary()

    # create analyzer
    analyzers = []
    for method in methods:
        analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
        # Some analyzers require training.
        #   analyzer.fit(test_img, batch_size=30, verbose=1)
        #  analyzers.append(analyzer)

    with open(('hipp_lrpCMP_resmodel_DELCODE_whole_ds.csv'), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(header)
        if (k==0):
            print('hipp_lrpCMP_resmodel_DELCODE_whole_ds.csv')
            print(header)

        for indx in range(len(grps)): # for all scans
            test_img = images[indx]
            test_img = np.reshape(test_img, (1,)+ test_img.shape) # add first subj index again to mimic original array structure

            #for method,analyzer in zip(methods, analyzers):
            a = analyzer.analyze(test_img, neuron_selection=1)
            np.clip(a, a_min=0, a_max=None, out=a) # set negative values to 0
            #a = np.squeeze(a)
            #a = scipy.ndimage.filters.gaussian_filter(a, sigma=0.8) # smooth activity image
            #scale = np.quantile(a, 0.99) # no need for abs(a)
            #if scale!=0:  # ignore if relevance maps contains only zeros, output will be zero as well
            #    a = (a/scale)

            rel_all = np.multiply(data_overlay, np.repeat(a, len(fname_overlay), axis=0))

            print([grps.RID.iloc[indx], ] + np.sum(rel_all, axis=(1,2,3,4)).tolist())
            csvwriter.writerow([grps.RID.iloc[indx], ] + np.sum(rel_all, axis=(1,2,3,4)).tolist())
        csvfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 100, 100, 120, 5)  140       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 50, 60, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 60, 5)     20        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 60, 5)     680       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 25, 25, 30, 5)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 30, 5)     20        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 25, 30, 5)     680       
__________

['3bcf0d9d2', 0.5405600070953369, 1.1013401746749878, 2.24007511138916, 0.07245495170354843, 0.3572555184364319, 1.6377201080322266, 1.2451421022415161, 1.412209391593933, 9.89984130859375]
['3f194bc00', 0.5028626918792725, 1.1377580165863037, 2.3641417026519775, 0.27049922943115234, 0.40822044014930725, 1.0162017345428467, 1.6008868217468262, 2.7229385375976562, 11.337112426757812]
['3f27b85a6', 0.7751905918121338, 0.6857616305351257, 1.3849871158599854, 0.23552510142326355, 0.309261292219162, 0.8269122242927551, 1.277334213256836, 2.34367036819458, 8.714945793151855]
['40a5f2347', 0.26436376571655273, 0.6746646761894226, 2.80521559715271, 0.2932068109512329, 0.5648965239524841, 2.524540424346924, 2.0778629779815674, 2.6627368927001953, 13.380558967590332]
['41fd5f400', 0.10119497030973434, 0.6529152393341064, 1.7061504125595093, 0.10539636760950089, 0.36349859833717346, 0.8286183476448059, 1.652911901473999, 1.3201355934143066, 8.104883193969727]
['4267264d6', 0.17506574094295502, 0.

['99d318d74', 0.28967270255088806, 0.8873078227043152, 1.686779499053955, 0.4392484426498413, 0.8206069469451904, 0.9295034408569336, 1.9274091720581055, 2.674934148788452, 10.675134658813477]
['9cf65bd61', 0.41803622245788574, 1.497188687324524, 2.7734298706054688, 0.09035804867744446, 0.42199990153312683, 2.222944974899292, 2.799879550933838, 2.6682045459747314, 14.345754623413086]
['9d9c4884f', 0.42357298731803894, 0.9531715512275696, 2.3369839191436768, 0.30057603120803833, 0.4455406069755554, 1.3509995937347412, 1.4836431741714478, 3.5065813064575195, 12.309257507324219]
['9ff13e297', 0.5762122869491577, 0.8678745031356812, 2.8466949462890625, 0.45038530230522156, 0.49428585171699524, 1.00064218044281, 1.6242716312408447, 4.648216247558594, 13.510506629943848]
['a092a7bfa', 0.9750986099243164, 1.975000023841858, 3.5574827194213867, 0.7122806906700134, 0.7400156259536743, 2.6540703773498535, 2.0992932319641113, 6.986545562744141, 23.048967361450195]
['a7618c667', 0.6931304931640625

['fdabdb866', 1.2836819887161255, 0.8469683527946472, 3.2494776248931885, 1.7750325202941895, 0.6574757695198059, 1.2896384000778198, 1.5711100101470947, 6.001636981964111, 17.643556594848633]
['ff6b0f91f', 0.2112051397562027, 1.0471688508987427, 1.69783353805542, 0.2716987431049347, 0.527097761631012, 0.8079254031181335, 0.9181339740753174, 1.673959732055664, 7.967533588409424]
['01c968c29', 0.4029572904109955, 1.1587905883789062, 1.4746497869491577, 0.1617373526096344, 0.3155587911605835, 1.147156000137329, 1.5296591520309448, 2.098046064376831, 9.954692840576172]
['02939fa93', 0.21858321130275726, 0.5594969987869263, 2.639914035797119, 0.07580558955669403, 0.551350474357605, 1.5801701545715332, 1.6003918647766113, 2.330749750137329, 10.83688735961914]
['0633efccb', 0.24421469867229462, 1.5637305974960327, 2.5433173179626465, 0.5999645590782166, 0.5665827393531799, 1.290576457977295, 1.7308709621429443, 5.758260250091553, 15.250442504882812]
['0675b30cb', 0.41894519329071045, 1.35775

['80146d3be', 0.404126912355423, 1.0269564390182495, 2.3387646675109863, 0.620067834854126, 0.5292841792106628, 1.8491734266281128, 2.3460333347320557, 4.157662868499756, 14.698349952697754]
['8240d1812', 0.950100302696228, 1.2086528539657593, 3.4067435264587402, 0.5569403767585754, 0.7241958379745483, 3.2887473106384277, 2.0904502868652344, 6.405734062194824, 20.245065689086914]
['86f85e91d', 0.15453603863716125, 0.8476203083992004, 2.257246732711792, 0.1877933293581009, 0.4282276928424835, 1.5148390531539917, 2.454618453979492, 2.6058359146118164, 11.398422241210938]
['881977bc8', 0.15143682062625885, 0.9990135431289673, 2.373288631439209, 0.2037413865327835, 0.29000604152679443, 1.4340142011642456, 1.174233078956604, 2.5071158409118652, 10.47014331817627]
['88571e8a9', 2.0070295333862305, 1.598358392715454, 2.121072292327881, 1.5613534450531006, 0.4960736334323883, 1.8246818780899048, 0.8701863288879395, 10.39591121673584, 23.249568939208984]
['8e1bd328f', 0.6304489970207214, 1.0236

...

['bc177df98', 1.8992365598678589, 2.156161069869995, 5.156379222869873, 4.222228050231934, 1.628550410270691, 2.9891371726989746, 4.485276222229004, 28.57421875, 52.6189079284668]
['c35ddeb62', 0.7200247645378113, 0.8118603229522705, 3.200218439102173, 1.649348497390747, 1.4727846384048462, 1.795186996459961, 2.122183084487915, 15.324254035949707, 27.91056251525879]
['c81213449', 0.7866083383560181, 0.979707658290863, 3.033510446548462, 1.3784492015838623, 0.7315990924835205, 1.8726493120193481, 1.6953606605529785, 10.947976112365723, 22.59195327758789]
['ca971f829', 2.0832009315490723, 0.9526493549346924, 3.7453534603118896, 0.9792405366897583, 0.9200122952461243, 2.513122797012329, 3.5565404891967773, 8.232059478759766, 25.892553329467773]
['cb0806279', 1.1510781049728394, 2.715569257736206, 5.040014743804932, 1.3913356065750122, 1.6004109382629395, 2.4948251247406006, 3.3374149799346924, 14.259029388427734, 33.88695526123047]
['d0887896f', 0.9574421644210815, 0.9681026339530945, 2.3

['4850fb0c1', 1.275534987449646, 1.0079854726791382, 6.438783645629883, 0.44766971468925476, 1.4960386753082275, 2.4545786380767822, 5.536549091339111, 12.464394569396973, 33.22207260131836]
['4bfeecc3c', 0.6707202792167664, 1.062423586845398, 2.718158721923828, 1.3846750259399414, 0.6256517767906189, 1.759599208831787, 2.5624070167541504, 8.475373268127441, 20.91714096069336]
['504f71ebe', 0.6211334466934204, 1.4492367506027222, 3.1585307121276855, 0.24626609683036804, 0.7962567806243896, 1.9786328077316284, 2.395576000213623, 2.451547861099243, 14.208246231079102]
['53199d407', 0.2226765751838684, 0.5139390230178833, 2.1511425971984863, 0.40175023674964905, 0.4558267891407013, 0.5412572026252747, 1.0407114028930664, 2.6881728172302246, 8.69884204864502]
['545f96b8d', 0.7568504810333252, 0.8430367708206177, 1.844690203666687, 1.554688572883606, 0.6494421362876892, 1.0931774377822876, 1.0818651914596558, 7.624454975128174, 15.516592979431152]
['54e0f059e', 0.34090983867645264, 1.118196

['b750d9dbe', 1.0474941730499268, 1.3119251728057861, 2.0886597633361816, 2.36092472076416, 0.508395791053772, 1.3885586261749268, 1.0527046918869019, 12.37925910949707, 22.403459548950195]
['bb1c45a71', 1.3919622898101807, 0.8257982730865479, 2.299844980239868, 0.8272287249565125, 0.521679699420929, 1.7775077819824219, 2.591623067855835, 9.597296714782715, 20.82764434814453]
['bc86f0ab2', 0.7633044719696045, 1.4184156656265259, 3.0778567790985107, 0.8500468134880066, 0.9603641629219055, 1.2759555578231812, 2.282289981842041, 6.843374252319336, 19.54253578186035]
['bcae977b3', 0.9773353338241577, 0.61195969581604, 1.2881938219070435, 1.107663631439209, 0.857868492603302, 1.6272960901260376, 1.171972632408142, 7.252686500549316, 16.223852157592773]
['bd6b65027', 0.2779025137424469, 0.9174599051475525, 2.3480923175811768, 0.2572331726551056, 0.43790265917778015, 1.1564775705337524, 1.9586191177368164, 5.550374984741211, 14.026198387145996]
['c31b2cac2', 1.357166051864624, 0.4885304272174

['abea7cf25', 0.5769014954566956, 0.7224263548851013, 2.1525015830993652, 3.2833669185638428, 0.4427851438522339, 1.415326476097107, 1.4209864139556885, 10.102720260620117, 19.16612434387207]
['ad0d19af0', 1.2001734972000122, 0.7298154830932617, 3.076214551925659, 3.1800732612609863, 1.3034405708312988, 1.80280601978302, 2.29559063911438, 14.677077293395996, 28.88542938232422]
['c54be1cb9', 1.017826795578003, 0.9022813439369202, 2.0962655544281006, 2.455146551132202, 0.48788994550704956, 1.6979966163635254, 1.1914162635803223, 9.045610427856445, 19.574337005615234]
['c6c639f6e', 1.4318110942840576, 1.1942875385284424, 2.3533191680908203, 1.641236662864685, 0.536529541015625, 1.4701601266860962, 1.9807910919189453, 8.617558479309082, 19.906408309936523]
['d350c3a5a', 1.3085932731628418, 2.1258583068847656, 9.234394073486328, 2.8348798751831055, 2.736729860305786, 2.8300952911376953, 5.990408897399902, 26.242002487182617, 55.30290985107422]
['d45929e58', 0.47083422541618347, 0.5916016101

['74def0e0c', 0.25564441084861755, 0.5655452013015747, 2.5787341594696045, 0.017580267041921616, 0.41465508937835693, 1.2897299528121948, 2.005518913269043, 0.9399372935295105, 9.629609107971191]
['766b96f80', 2.9487574100494385, 2.275932788848877, 7.461661338806152, 6.522948265075684, 2.284590005874634, 2.124575614929199, 4.741942405700684, 24.835309982299805, 52.726341247558594]
['7a14bdf5d', 0.9368210434913635, 0.6820942759513855, 2.0401785373687744, 0.35940584540367126, 0.5762167572975159, 2.1262075901031494, 2.638400077819824, 14.233871459960938, 24.791866302490234]
['85eb9b1a2', 1.0800764560699463, 0.9442678689956665, 2.1999382972717285, 2.7983572483062744, 1.1840466260910034, 1.996009349822998, 2.1390867233276367, 21.36086082458496, 34.339256286621094]
['8e66edf5e', 0.5644193291664124, 1.2611290216445923, 3.9561808109283447, 0.8848553895950317, 1.1578160524368286, 2.6984779834747314, 2.581468343734741, 8.68883991241455, 23.75210952758789]
['90af3fc23', 0.7997357249259949, 1.4093